In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls drive/colab

checkpoints   lab1.ipynb  models.py	   test_labels.npy   train_labels.npy
data_samples  layers.py   test_images.npy  train_images.npy  vgg_16.ckpt


In [0]:
import tensorflow as tf
# print(tf.__version__)
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import time
import os
from PIL import Image

import sys
sys.path.insert(0,'drive/colab')
from layers import *
from models import *

In [0]:
# Parameters
BATCH_SIZE = 8
EPOCHES = 1000
SAVE_EVERY = 100
INITIAL_LR = 0.001
REDUCE_LR_EVERY = 200
TEST_EVERY = 10
SHOW_EVERY = 10


In [10]:
print('Loading Data...')
num_train_samples = 2805
train_images = np.load('drive/colab/train_images.npy')
train_labels = np.load('drive/colab/train_labels.npy')
num_test_samples = 108
test_images = np.load('drive/colab/test_images.npy')
test_labels = np.load('drive/colab/test_labels.npy')
palette = Image.open('drive/colab/data_samples/2008_003546_cls.png').getpalette()
print('Loaded')

Loading Data...
Loaded


In [0]:
# Placeholders
inputs = tf.placeholder(tf.float32, shape=[None, 128, 128, 3])
labels = tf.placeholder(tf.int32, shape=[None, 128, 128])
lr = tf.placeholder(tf.float32, shape=[])

In [0]:
# Network: Fully convolutional netework simplified
sm_inputs = SubstractMean(inputs, mean=[123.68, 116.78, 103.94])
with tf.variable_scope('vgg_16') as scope:
    vgg12, vgg22, vgg33, vgg43, vgg54 = vgg_16(sm_inputs)

with tf.variable_scope('FCN') as scope:
    logits = FCN(vgg54, BATCH_SIZE)

In [0]:
# Prediction
outputs = tf.argmax(tf.nn.softmax(logits, axis=-1), axis=-1)
miou = tf.metrics.mean_iou(labels=labels, predictions=outputs, num_classes=21)

# Loss: Cross-entropy
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

In [0]:
# Optimizer: ADAM
fcn_params = [v for v in tf.trainable_variables() if v.name.startswith('FCN/')]
vgg_params = [v for v in tf.trainable_variables() if v.name.startswith('vgg_16/')]

train_op = tf.train.AdamOptimizer(lr).minimize(loss, var_list=fcn_params)

In [0]:
# Restore op for pre-trained weight for VGG16
def name_in_checkpoint(var):
    if "W" in var.op.name:
        return var.op.name.replace("W", "weights")
    if "b" in var.op.name:
        return var.op.name.replace("b", "biases")
variables_to_restore = {name_in_checkpoint(var):var for var in vgg_params}
restorer = tf.train.Saver(variables_to_restore)

# TF saver
saver = tf.train.Saver()

In [0]:
# TF Session
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    # load vgg weight
    restorer.restore(sess, "drive/colab/vgg_16.ckpt")

#     # if you want resume training:
#     saver.restore(sess, 'drive/colab/checkpoints/FCN_Epoch_2.ckpt')
    
    # Training
    curr_lr = INITIAL_LR
    for e in range(EPOCHES):
        running_loss = 0
        running_miou = 0
        running_time = 0
        # order = np.arange(num_train_samples)
        # np.random.shuffle(order)
        # train_images = train_images[order]
        # train_labels = train_labels[order]
        for i, p in enumerate(range(0, num_train_samples-BATCH_SIZE, BATCH_SIZE)):
            tt = time.time()
            batch_images = train_images[p:p+BATCH_SIZE].astype(np.float32)
            batch_labels = train_labels[p:p+BATCH_SIZE].astype(np.int32)

            _miou, _outputs, _loss,  _= sess.run([miou, outputs, loss, train_op], feed_dict={inputs: batch_images, labels: batch_labels, lr:curr_lr})
            rT = time.time() - tt
            running_loss += np.mean(_loss)
            running_miou += np.mean(_miou[0])
            running_time += rT
       
        print('Epoch: {:3d} | Iter: {:4d} | Loss: {:4.3e} | mIoU: {:3.4f} | Seconds/iter: {:4.3f}s'.format(e+1, i, running_loss/(i+1), 100*running_miou/(i+1), running_time/(i+1)))

        if (e+1) % SAVE_EVERY == 0:
            save_path = saver.save(sess, 'drive/colab/checkpoints/FCN_Epoch_'+str(e+1)+'.ckpt')
            print("Model saved in file: %s" % save_path)
            
        if (e+1) % REDUCE_LR_EVERY == 0:
            curr_lr *= 0.1

            
        # Show
        if (e+1) % SHOW_EVERY == 0: 
            batch_images = test_images[0:8].astype(np.float32)
            batch_labels = test_labels[0:8].astype(np.int32)
            _outputs = sess.run(outputs, feed_dict={inputs: batch_images, labels: batch_labels})
            
            plt.figure(0)
            for b in range(8):
                img = Image.fromarray((batch_images[b]).astype(np.uint8))
                lab = Image.fromarray(batch_labels[b].astype(np.uint8))
                lab.putpalette(palette)
                pred = Image.fromarray(_outputs[b].astype(np.uint8))
                pred.putpalette(palette)

                plt.subplot(3,8,1+b); plt.imshow(img); plt.axis('off')
                plt.subplot(3,8,9+b); plt.imshow(lab); plt.axis('off')
                plt.subplot(3,8,17+b); plt.imshow(pred); plt.axis('off')
            plt.show()
                
                
        # Test
        if (e+1) % TEST_EVERY == 0: 
            test_miou = 0
            for i, p in enumerate(range(0, num_test_samples-BATCH_SIZE, BATCH_SIZE)):
                batch_images = train_images[p:p+BATCH_SIZE].astype(np.float32)
                batch_labels = train_labels[p:p+BATCH_SIZE].astype(np.int32)
                _miou = sess.run(miou, feed_dict={inputs: batch_images, labels: batch_labels})
                test_miou += np.mean(_miou[0])
            print('[TEST] Epoch: {:3d} | mIoU: {:3.4f}'.format(e+1, 100*test_miou/(i+1)))

                

INFO:tensorflow:Restoring parameters from drive/colab/vgg_16.ckpt
Epoch:   1 | Iter:  349 | Loss: 1.164e+00 | mIoU: 18.0911 | Seconds/iter: 0.066s
Epoch:   2 | Iter:  349 | Loss: 8.889e-01 | mIoU: 26.8299 | Seconds/iter: 0.062s
